Holder Table Function 
=====================


In [11]:
using Pkg
Pkg.activate("../../.")
using Globtim
# include("../../../src/lib_func.jl") # Include the library of functions

# Constants and Parameters
d = 2 # Initial Degree 
const n, a, b = 2, 10, 1 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .5 , 1 / 10  # Sampling parameters
const tol_l2 = 1.5e-2            # Define the tolerance for the L2-norm

f = HolderTable # Objective function

  Activating project at `~/Globtim.jl`


HolderTable (generic function with 1 method)

In [12]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        println("Current degree: ", d)
        global d += 1
    end
end;

current L2-norm: 0.3225768101075613
Number of samples: 8
Current degree: 2
current L2-norm: 0.34164522984081835
Number of samples: 12
Current degree: 3
current L2-norm: 0.15420669778631205
Number of samples: 17
Current degree: 4
current L2-norm: 0.16150972067976094
Number of samples: 22
Current degree: 5
current L2-norm: 0.10982203454764894
Number of samples: 28
Current degree: 6
current L2-norm: 0.09002980177601799
Number of samples: 34
Current degree: 7
current L2-norm: 0.07219154289178054
Number of samples: 41
Current degree: 8
current L2-norm: 0.061872080671082644
Number of samples: 48
Current degree: 9
current L2-norm: 0.0424716817826579
Number of samples: 56
Current degree: 10
current L2-norm: 0.03650326901605909
Number of samples: 65
Current degree: 11
current L2-norm: 0.026535168562028325
Number of samples: 74
Current degree: 12
current L2-norm: 0.02370545610011522
Number of samples: 83
Current degree: 13
current L2-norm: 0.018346228636119927
Number of samples: 93
Current degre

In [13]:
using DynamicPolynomials, DataFrames
using HomotopyContinuation, ProgressLogging

@polyvar(x[1:n]) # Define polynomial ring 
pol = main_nd(x, n, d, poly_approx.coeffs) 

println("The polynomial is: ", pol)
grad = differentiate.(pol, x)
sys = System(grad)
println("The system is of degree: ", d-1)

The polynomial is: -272287136774817571//1152921504606846976 + 108604667564791267587//1298074214633706907132624082305024*x[2] + 51546911033889635303//2596148429267413814265248164610048*x[1] + 21875266918444178683//576460752303423488*x[2]^2 - 38284420041259286213//5192296858534827628530496329220096*x[1]*x[2] - 51485708243114346853//576460752303423488*x[1]^2 - 395674829397065464791//162259276829213363391578010288128*x[2]^3 + 76400145264082167513//1298074214633706907132624082305024*x[1]*x[2]^2 - 175761119289352194891//649037107316853453566312041152512*x[1]^2*x[2] - 441271482591076931629//649037107316853453566312041152512*x[1]^3 - 60974340487694403993//72057594037927936*x[2]^4 + 88351814539797080555//649037107316853453566312041152512*x[1]*x[2]^3 - 20543039454962597891//288230376151711744*x[1]^2*x[2]^2 + 69289945661599678741//1298074214633706907132624082305024*x[1]^3*x[2] + 145754934449090454647//72057594037927936*x[1]^4 + 1664234892650102350865//81129638414606681695789005144064*x[2]^5 - 971

In [14]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false);

Tracking 225 paths...   1%|▎                            |  ETA: 0:19:47



Tracking 225 paths...  18%|█████▏                       |  ETA: 0:00:50



Tracking 225 paths...  28%|████████▏                    |  ETA: 0:00:28



Tracking 225 paths...  45%|█████████████                |  ETA: 0:00:14



Tracking 225 paths...  61%|█████████████████▊           |  ETA: 0:00:07



Tracking 225 paths...  81%|███████████████████████▌     |  ETA: 0:00:03



Tracking 225 paths... 100%|█████████████████████████████| Time: 0:00:11
  # paths tracked:                  225
  # non-singular solutions (real):  225 (189)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         225 (189)


In [15]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

In [16]:
using GLMakie
# Extract coordinates and function values
coords = poly_approx.scale_factor * poly_approx.grid
z_coords = poly_approx.z;


In [17]:
if size(coords)[2] == 2  # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Holder Table Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with a darker core
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=:black, label="Sampled Data Core")

    # Plot the sampled data with adjusted size and color to create a halo effect
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=4, color=z_coords, colormap=:viridis, label="Sampled Data Halo")

    # Plot the exact approximant critical points with adjusted size
    scatter!(ax, df.x, df.y, df.z, markersize=10, color=:red, label="Exact approximant critical points")

    display(fig)
end

GLMakie.Screen(...)

Should plot the polynomial approximant too.

In [9]:
# Update tolerance
const tol_l2 = 1.5e-3

# Re-run the main computation loop
d = 2 # Reset degree
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        println("Current degree: ", d)
        global d += 1
    end
end;

# Re-run the polynomial system setup and solve
pol = main_nd(x, n, d, poly_approx.coeffs) 
grad = differentiate.(pol, x)
sys = System(grad)
println("The system is of degree: ", d-1)

Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false)

# Filter points and create DataFrame
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Collect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

current L2-norm: 0.3225768101075613


Number of samples: 8
Current degree: 2
current L2-norm: 0.34164522984081835
Number of samples: 12
Current degree: 3
current L2-norm: 0.15420669778631205
Number of samples: 17
Current degree: 4
current L2-norm: 0.16150972067976094
Number of samples: 22
Current degree: 5
current L2-norm: 0.10982203454764894
Number of samples: 28
Current degree: 6
current L2-norm: 0.09002980177601799
Number of samples: 34
Current degree: 7
current L2-norm: 0.07219154289178054
Number of samples: 41
Current degree: 8
current L2-norm: 0.061872080671082644
Number of samples: 48
Current degree: 9
current L2-norm: 0.0424716817826579
Number of samples: 56
Current degree: 10
current L2-norm: 0.03650326901605909
Number of samples: 65
Current degree: 11
current L2-norm: 0.026535168562028325
Number of samples: 74
Current degree: 12
current L2-norm: 0.02370545610011522
Number of samples: 83
Current degree: 13
current L2-norm: 0.018346228636119927
Number of samples: 93
Current degree: 14
current L2-norm: 0.0166368349

InterruptException: InterruptException:

In [10]:
# Plot using GLMakie

# Extract coordinates and function values
coords = poly_approx.scale_factor * poly_approx.grid
z_coords = poly_approx.z

if size(coords)[2] == 2  # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Mixture of Gaussians Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with adjusted size and color to create a halo effect
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=z_coords, colormap=:viridis, label="Sampled Data Halo")

    # Plot the exact approximant critical points with adjusted size
    scatter!(ax, df.x, df.y, df.z, markersize=10, color=:red, label="Exact approximant critical points")

    display(fig)
end

GLMakie.Screen(...)